Библиотеки

In [ ]:
# Работа с df
import pandas as pd
# Парсинг
from bs4 import BeautifulSoup as bs
import urllib
import re
import requests as rq
import time

In [ ]:
df = pd.read_json("data.json")
all_links = df['link'].tolist()

Ссылки на страницы отзывов

In [ ]:
# уникальные названия 419 отелей
name_parts = []
def get_part(surl):
    url = urllib.parse.urlparse(surl)
    lurl = list(url)
    for_regex = lurl[2]
    start = re.sub(r'/hotel/ie/', '', for_regex)
    end = re.sub(r'.en-gb.html', '', start)
    name_parts.append(end)
for link in all_links:
    get_part(link)


# ссылки на первую страницу каждого из 419 отелей
links_for_reviews = []
def get_a_rwlink(name):
    links_for_reviews.append("https://www.booking.com/reviewlist.html?aid=304142&label=gen173nr-1FCAsoaUIPYmVyZXNmb3JkLWhvdGVsSDNYBGjCAYgBAZgBMbgBB8gBDNgBAegBAfgBAogCAagCA7gC79TQngbAAgHSAiRiN2JlOTc5MC1lZTc0LTQxODYtOTU2MC0wMDU1ZWExZGM1ZjHYAgXgAgE&sid=f52429649313d6700f51428e2f0d275f&cc1=ie&dist=1&pagename=" + name +"&type=total&rows=10&_=1674848881427&offset=0")
for name in name_parts:
    get_a_rwlink(name)

# заголовки и отзывы
reviews_titles = []
reviews = []
def get_reviews_titles(url):
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    page = rq.get(url, headers=headers)
    soup = bs(page.content,'lxml') 
    soup_reviews_titles = soup.find_all("h3", class_=" c-review-block__title c-review__title--ltr ")
    soup_reviews = soup.find_all("span", class_="c-review__body")
    for title in soup_reviews_titles:
        reviews_titles.append(title.get_text().replace('\n', ''))
    for review in soup_reviews:
        reviews.append(review.get_text().replace('\r', ''))
for i in links_for_reviews:
    get_reviews_titles(i)

Удаляем отзывы-дубли

In [ ]:
def remove_duplicates(reviews):
    unique_reviews = set()
    duplicates = []
    for review in reviews:
        if review in unique_reviews:
            duplicates.append(review)
        else:
            unique_reviews.add(review)
    unique_reviews = list(unique_reviews)
    return unique_reviews, duplicates
unique_reviews, duplicates = remove_duplicates(reviews)
#print(unique_reviews)
#print(duplicates)

Сохраняем

In [ ]:
df = pd.DataFrame(reviews, columns = ["review"])
df.to_json('reviews.json')
df.to_csv('reviews.csv')
df = pd.DataFrame(links_for_reviews, columns = ["link"])
df.to_json('links_for_reviews.json')
df.to_csv('links_for_reviews.csv')
df = pd.DataFrame(reviews_titles, columns = ["title"])
df.to_json('reviews_titles.json')
df.to_csv('reviews_titles.csv')